## 从key文件读取key值
*输入参数: file_path
文件路径
*返回值: 存有key的列表

In [4]:
def read_keys_from_file(key_path):
    key_pool = []
    with open(key_path, 'r') as file:
        for line in file:
            key = line.strip()  # 去掉每一行的换行符
            key_pool.append(key)  # 将每一行存入key池中
    return key_pool

In [5]:
# 读取key
key_path = "key.txt"  # 将这个替换为你的txt文件的路径
key_pool = read_keys_from_file(key_path)

In [6]:
import os
from random import random
os.environ['HTTP_PROXY'] = "127.0.0.1:33210"
os.environ['HTTPS_PROXY'] = "127.0.0.1:33210"
os.environ["OPENAI_API_KEY"] = key_pool[1]

from langchain.document_loaders import PyPDFLoader

from PyPDF2 import PdfMerger
file_list = ["KT820说明书V2.0 201909.pdf","GSK980TC3系列  编程操作手册（2018年6月第3版）0622定(1).pdf"]
file_merger = PdfMerger()       # 合并pdf文件
for pdf in file_list:
    file_merger.append(pdf)
# 合并后的pdf放在本地
file_merger.write("merge.pdf")
loader = PyPDFLoader("merge.pdf")
unprocessed_documents = loader.load()


In [56]:
# 用户输入的问题
question = "100-100*99%12等于多少"

import jieba
# 利用jieba_fast进行分词
def jieba_extract_keywords(question):
    # 使用jieba进行分词
    keywords = jieba.cut(question)
    return keywords

# 利用chatgpt进行分词
def gpt_extract_keywords(question):

    from langchain.prompts import ChatPromptTemplate
    prompts = f"""
            ----
            {question}
            ----
            注意如果question是数学表达式或包含数学表达式，不需要提取数学表达式中的关键词而是直接返回完整的数学表达式。
            你是一个专家级的文本关键词提取大师，根据上面的question，从其中提取关键词并一行一行地返回它们.关键词提取应该尽可能全面，包括单个有意义的词汇和有意义的词组。

            下面这个是正确示范，仅作为参考，不用返回:
            例子1
            question:1+1*99
            正确输出:
            1+1*99

            例子2
            question: 请告诉我G100代码有哪些功能
            正确输出:
            G100
            代码
            G100代码
            功能

            例子3
            question: 报警号197是什么
            正确输出:
            报警号
            197
            报警号197

            例子4
            question: 使用手册的版权归谁所有
            错误输出:
            使用手册的版权
            归谁所有
            正确输出：
            使用手册
            版权
            使用手册的版权
            归谁所有

            例子5
            question: GSK980TC3的报警号0001是什么
            正确输出：
            GSK980TC3
            报警号
            0001
            报警号0001


    """

    promptsTemplate = ChatPromptTemplate.from_template(prompts)
    from langchain.chains import LLMChain
    from langchain.chat_models import ChatOpenAI
    llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=0)
    chain = LLMChain(llm=llm, prompt=promptsTemplate)
    response = chain.run(question=question)
    return response

In [57]:
import PyPDF2
def search_keywords_in_pdf(doc,keywords):
    keyword_count_per_page = {}
    for page_num in range(len(doc)):
        # 计算当前页面的关键词出现次数
        count = sum(doc[page_num].page_content.count(keyword) for keyword in keywords)

        # 如果关键词出现次数大于0，则记录该页码和次数
        if count > 0:
            keyword_count_per_page[page_num+1] = count
    # 根据关键词出现次数从大到小排序页面号
    sorted_page_numbers = [k for k, v in sorted(keyword_count_per_page.items(), key=lambda item: item[1],reverse=True)]
    return sorted_page_numbers

def get_page_content(file_path, page_number):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        # 确保页面号在有效范围内
        if page_number - 1 < len(reader.pages):
            page = reader.pages[page_number - 1]
            text = page.extract_text()
            return text
        else:
            return "指定的页码超出了PDF文件的范围"


In [58]:
import time

start_time = time.time()
# 从问题中提取关键词
keywords = gpt_extract_keywords(question)
print(keywords)
import ast
keywords = keywords.splitlines()

print(keywords)
print(type(keywords))

100-100*99%12
['100-100*99%12']
<class 'list'>


In [59]:

os.environ["OPENAI_API_KEY"] = key_pool[2]
from langchain.chat_models import ChatOpenAI


In [60]:
pages = search_keywords_in_pdf(unprocessed_documents,keywords)
print(f"关键词匹配的页面有: {pages}")

关键词匹配的页面有: []


In [61]:
documents = []
pages = pages[:50]

from collections import defaultdict
# 使用 defaultdict 存储未处理的文档，键为页面编号，值为文档列表
unprocessed_documents_dict = defaultdict(list)
for doc in unprocessed_documents:
    unprocessed_documents_dict[doc.metadata['page']].append(doc)

# 使用列表推导式来按页面顺序添加文档
documents = [doc for page in pages for doc in unprocessed_documents_dict[page - 1] if page - 1 in unprocessed_documents_dict]

print(documents)
print(len(documents))

[]
0


In [62]:
from langchain.text_splitter import CharacterTextSplitter
os.environ["OPENAI_API_KEY"] = key_pool[3]


text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()

from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0613"), chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

prompt = f"""
    下面是用户提的问题,以'''为分割符号
    '''
    {question}
    '''
    回答要求语气温和有礼貌、并且回答内容要详细、有具体文本支撑观点。
"""

res = qa(prompt)
answer, docs = res['result'], res['source_documents']
end_time = time.time()
execution_time = end_time - start_time
print(f"代码运行时间: {execution_time} 秒")
print(answer)


代码运行时间: 11.736984729766846 秒
根据提供的上下文，我们可以使用用户宏程序功能来计算这个表达式。根据用户宏程序功能的说明，我们可以使用以下运算代码来计算表达式：

#i = #j - trunc(#j / #k) × #k

其中，#j表示100，#k表示99，我们可以将表达式转化为以下形式：

#i = 100 - trunc(100 / 99) × 99

根据计算规则，我们先计算100 / 99，得到一个小数，然后使用trunc函数将小数部分舍去，最后再乘以99。根据计算结果，我们可以得到：

#i = 100 - trunc(1.010101010101010101010101010101 × 99) × 99


